In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
ATLAS_URI = os.getenv('ATLAS_URI')
TOGETHER_API_KEY = os.getenv('TOGETHER_API_KEY')

if not (ATLAS_URI or TOGETHER_API_KEY):
    raise Exception ("'ATLAS_URI' or 'TOGETHER_API_KEY' is not set. Please set it above to continue...")
print("Fetched Env. Variables")

Fetched Env. Variables


In [3]:
DB_NAME = 'stackup_zendesk'
COLLECTION_NAME = 'data'
INDEX_NAME = 'idx_embedding'

In [4]:
import pymongo

mongodb_client = pymongo.MongoClient(ATLAS_URI)
print ("Atlas client initialized")

Atlas client initialized


In [5]:
from langchain_together.embeddings import TogetherEmbeddings

embed_model = TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval", api_key=TOGETHER_API_KEY)

In [6]:
from llama_index.core import ServiceContext

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

/tmp/ipykernel_68123/1756614690.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)


LLM is explicitly disabled. Using MockLLM.


In [7]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext

vector_store = MongoDBAtlasVectorSearch(
    mongodb_client = mongodb_client,
    db_name = DB_NAME, collection_name = COLLECTION_NAME,
    vector_index_name  = 'idx_embeddings',
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [8]:
%%time
from llama_index.core import SimpleDirectoryReader

data_dir = 'cleaned_data'

docs = SimpleDirectoryReader(
        input_dir=data_dir
).load_data()

print (f"Loaded {len(docs)} chunks from '{data_dir}'")

Loaded 1 chunks from 'cleaned_data'
CPU times: user 11.9 ms, sys: 3 μs, total: 11.9 ms
Wall time: 11.6 ms


In [9]:
%%time
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    docs, 
    storage_context=storage_context,
    service_context=service_context,
)

CPU times: user 288 ms, sys: 7.1 ms, total: 296 ms
Wall time: 25 s
